In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time

In [2]:
def wait(s):
    time.sleep(s)

In [3]:
def call(f, *args):
    s = "(" + ", ".join(map(str, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except BadError:
            raise
        except Exception:
            raise
        else:
            return rv

In [4]:
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

In [5]:
h = HelloApp('192.168.1.21')

In [6]:
call(h.setag, 0, 10)

Calling setag(0, 10)
